In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seg-train/seg_train/mountain/14986.jpg
/kaggle/input/seg-train/seg_train/mountain/3138.jpg
/kaggle/input/seg-train/seg_train/mountain/1700.jpg
/kaggle/input/seg-train/seg_train/mountain/16257.jpg
/kaggle/input/seg-train/seg_train/mountain/2863.jpg
/kaggle/input/seg-train/seg_train/mountain/771.jpg
/kaggle/input/seg-train/seg_train/mountain/12167.jpg
/kaggle/input/seg-train/seg_train/mountain/17643.jpg
/kaggle/input/seg-train/seg_train/mountain/6560.jpg
/kaggle/input/seg-train/seg_train/mountain/10162.jpg
/kaggle/input/seg-train/seg_train/mountain/4009.jpg
/kaggle/input/seg-train/seg_train/mountain/15823.jpg
/kaggle/input/seg-train/seg_train/mountain/820.jpg
/kaggle/input/seg-train/seg_train/mountain/6272.jpg
/kaggle/input/seg-train/seg_train/mountain/15783.jpg
/kaggle/input/seg-train/seg_train/mountain/3446.jpg
/kaggle/input/seg-train/seg_train/mountain/11034.jpg
/kaggle/input/seg-train/seg_train/mountain/6295.jpg
/kaggle/input/seg-train/seg_train/mountain/5018.jpg
/kaggl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from keras import layers, Sequential, losses
import sys
import os

from sklearn.utils import shuffle
import cv2
import glob
import zipfile
from matplotlib import pyplot as plt
import pickle
from keras_preprocessing.image import array_to_img
import pathlib
from zipfile import ZipFile

from PIL import Image
import os, sys
import cv2
import random
from tensorflow import keras 
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
#from imutils import paths

from tensorflow.keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import tensorflow_datasets as tfds
import PIL
import PIL.Image

In [3]:
train_path = '../input/seg-train/seg_train' 
test_path = '../input/testzip'

# 1

# new

In [4]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_path,
                                                 shuffle=True,
                                                 target_size=(150,150), 
                                                 subset="training",
                                                 class_mode='categorical')

val_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory=train_path,
                                                 shuffle=True,
                                                 target_size=(150,150), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.


In [5]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal",input_shape=(150,150,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

2022-11-07 17:55:03.472180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 17:55:03.565325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 17:55:03.566094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 17:55:03.567834: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
#2
base_model = tf.keras.applications.DenseNet169(include_top=False, weights="imagenet", 
                                                   input_tensor=Input(shape=(150, 150, 3)))

inputs = layers.Input(shape = (150,150, 3), name = 'input_layer')
x = data_augmentation(inputs)
x = base_model(x, training = False)
x = layers.GlobalAveragePooling2D(name = 'global_avg_pool_layer')(x)
outputs = layers.Dense(6, activation = 'softmax', name = 'output_layer')(x)
model = tf.keras.Model(inputs, outputs)

51888128/51877672 [==============================] - 0s 0us/step


In [7]:
#1
base_model = tf.keras.applications.DenseNet169(include_top=False, weights="imagenet", 
                                                   input_tensor=Input(shape=(150, 150, 3)))

In [8]:
x = base_model.output

x = Flatten(name='flatten')(x)

x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(36, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

In [9]:
sgd = SGD(learning_rate=0.0001, momentum=0.9, decay=0.0001,)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics = ['accuracy'])


In [10]:
# DenseNet169 1 not agument
history = model.fit(train_data, epochs=20, batch_size=32, validation_data = val_data)

2022-11-07 17:55:15.725942: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-11-07 17:55:30.149137: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


351/351 [==============================] - 88s 190ms/step - loss: 0.8013 - accuracy: 0.7077 - val_loss: 0.3273 - val_accuracy: 0.8909
Epoch 2/20
351/351 [==============================] - 59s 169ms/step - loss: 0.3295 - accuracy: 0.8915 - val_loss: 0.2512 - val_accuracy: 0.9162
Epoch 3/20
351/351 [==============================] - 59s 168ms/step - loss: 0.2505 - accuracy: 0.9181 - val_loss: 0.2326 - val_accuracy: 0.9208
Epoch 4/20
351/351 [==============================] - 63s 179ms/step - loss: 0.1924 - accuracy: 0.9365 - val_loss: 0.2344 - val_accuracy: 0.9205
Epoch 5/20
351/351 [==============================] - 60s 171ms/step - loss: 0.1610 - accuracy: 0.9472 - val_loss: 0.2285 - val_accuracy: 0.9244
Epoch 6/20
351/351 [==============================] - 59s 168ms/step - loss: 0.1334 - accuracy: 0.9566 - val_loss: 0.2303 - val_accuracy: 0.9240
Epoch 7/20
351/351 [==============================] - 59s 168ms/step - loss: 0.1030 - accuracy: 0.9686 - val_loss: 0.2392 - val_accuracy: 0.9

In [11]:
# image_generator 2 agument
history = model.fit(train_data, epochs=20, batch_size=32, validation_data = val_data)

Epoch 1/20
351/351 [==============================] - 58s 165ms/step - loss: 0.0269 - accuracy: 0.9926 - val_loss: 0.2871 - val_accuracy: 0.9230
Epoch 2/20
351/351 [==============================] - 59s 168ms/step - loss: 0.0248 - accuracy: 0.9930 - val_loss: 0.2902 - val_accuracy: 0.9265
Epoch 3/20
351/351 [==============================] - 59s 168ms/step - loss: 0.0251 - accuracy: 0.9933 - val_loss: 0.2936 - val_accuracy: 0.9244
Epoch 4/20
351/351 [==============================] - 60s 171ms/step - loss: 0.0230 - accuracy: 0.9938 - val_loss: 0.2898 - val_accuracy: 0.9272
Epoch 5/20
351/351 [==============================] - 64s 181ms/step - loss: 0.0230 - accuracy: 0.9937 - val_loss: 0.2999 - val_accuracy: 0.9240
Epoch 6/20
351/351 [==============================] - 59s 169ms/step - loss: 0.0216 - accuracy: 0.9949 - val_loss: 0.2915 - val_accuracy: 0.9283
Epoch 7/20
351/351 [==============================] - 59s 169ms/step - loss: 0.0207 - accuracy: 0.9950 - val_loss: 0.2981 - val_ac

In [12]:
image_generator_test = ImageDataGenerator(rescale=1/255)    

test_data = image_generator_test.flow_from_directory(batch_size=32,
                                                 directory=test_path,
                                                 shuffle=True,
                                                 target_size=(150,150))

Found 3000 images belonging to 1 classes.


In [13]:
predictions = model.predict(test_data)

In [14]:
predictions

array([[4.9203148e-09, 6.0145871e-09, 8.9419973e-07, 9.9999905e-01,
        6.2495245e-11, 2.1815376e-08],
       [5.4463499e-06, 1.4076110e-07, 1.0965538e-07, 2.4083073e-07,
        2.1111366e-07, 9.9999380e-01],
       [6.1604027e-05, 2.2102421e-04, 5.0308616e-03, 9.9453884e-01,
        9.7855464e-06, 1.3786329e-04],
       ...,
       [9.9999571e-01, 2.8616969e-06, 6.0517186e-07, 7.4458351e-09,
        6.8034433e-07, 1.8804919e-07],
       [5.0014702e-08, 9.9999475e-01, 1.5836734e-07, 3.3779408e-08,
        2.3863365e-07, 4.7496706e-06],
       [4.8606690e-08, 1.0362266e-06, 9.9997866e-01, 1.9847812e-05,
        4.2661719e-07, 1.2293050e-08]], dtype=float32)

In [15]:
pred_labels = np.argmax(predictions, axis=1)

In [16]:
pred_labels

array([3, 5, 3, ..., 0, 1, 2])

In [17]:
x = pred_labels.tolist()

In [18]:
label = [ int(f.split("/")[-1].split('.')[0]) for f in test_data.filenames]

In [19]:
submission = pd.DataFrame({'id':label, 'label':x})
submission

,id,label
0,0,3
1,1,5
2,10,3
3,100,1
4,1000,4
...,...,...
2995,995,0
2996,996,5
2997,997,0
2998,998,1


In [20]:
submission= submission.sort_values(by=['id'], ascending=True)
submission

,id,label
0,0,3
1,1,5
1112,2,0
2223,3,4
2334,4,2
...,...,...
2218,2995,2
2219,2996,5
2220,2997,2
2221,2998,5


In [21]:
from pathlib import Path  
filepath = Path('out7.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission.to_csv(filepath,index=False)